In [1]:
import pickle
from ownership import *

In [3]:
with open('data/tx_rows.p', 'rb') as f:
    tx_rows = pickle.load(f)

Let's try filtering on events instead of transactions. It's more straightforward to write a simulator for PepeDapp using *just* the CardsTransferred event!

First, I'm going to compare the keccak hash used in topics[0] for the *first* PepeDapp tx to sanity check my mechanism for finding events.

In [42]:
from Crypto.Hash import keccak
keccak_hash = keccak.new(digest_bits=256)
keccak_hash.update(b'CardCreated(bytes32,uint256)')
test_val = keccak_hash.hexdigest()
val_from_bq = '5ac4a187d29c4350cc4df530b82f63c563573f6a54125942974ca8c99e4bede9'
test_val == val_from_bq

True

In [45]:
cards_transferred_hash = keccak.new(digest_bits=256)
cards_transferred_hash.update(b'CardsTransferred(bytes32,address,address,uint256)')
'0x' + cards_transferred_hash.hexdigest()

'0xca854e9a512debe5ebf4a5228b275f9d8c3c005f95ba3660d7a6a66fdc72b112'

This is the value that I want to query with in the BigQuery event logs table. 

Querying done! Now, let's parse this data properly, particularly the other topics in each event log.

Having looked at the logs manually, it looks like:
- tx[1] == sig
- tx[2] == fromAddr
- tx[3] == toAddr
- data == count (as hex str though)

In [2]:
import json
with open('data/events.json', 'r') as f:
    logs_data = json.load(f)
    
transfers = []
for log in logs_data:
    sig = log['topics'][1]
    from_addr = Web3.toChecksumAddress('0x' + log['topics'][2][26:])
    to_addr = Web3.toChecksumAddress('0x' + log['topics'][3][26:])
    count = int(log['data'], 0)
    transfers.append(Transfer(sig, from_addr, to_addr, count))

Ok, let's go back to creating the initial state. Don't need to keep track of sales anymore!

In [3]:
sigs = get_all_sigs()
ownership = {ADMIN_ADDRESS: {}}
for sig in sigs:
    count = get_sig_count(sig)
    ownership[ADMIN_ADDRESS]['0x' + sig.hex()] = count
INITIAL_STATE = PepeState(ownership)

In [4]:
count = 0
cur_state = INITIAL_STATE
for transfer in transfers:
    cur_state = process_transfer(cur_state, transfer)
    count += 1

Hells yes! Let's create a single CSV, sorted by (sig, address) of the ownership map:

In [8]:
cur_state.ownership['0xcB46219bA114245c3A18761E4f7891f9C4BeF8c0']

{'0x91781647437c48bc328f25d4c47b6f24cf8a46ab1b0f3c63704f9705178c0b5d': 4,
 '0x2fde0bc20c20a82166355cf1b6b78d01a4411d4794703642b04d3ec0232548ca': 26,
 '0xfa61d833ca2c8fc6f23aabbc18a23eef4e6313bc47446ecb06dbb148eefe349a': 1,
 '0x7f43c4e20a98081432b88599fcdbf9e2d8a3b7093515a473f828190a2d82e045': 9,
 '0x4fc4fd5e45256c6074fcfa6705c9f8f17cca4fbc2e334c238a245edecc09fabf': 6,
 '0x10c7871c5794b23562937b3b10abe535d5164acfee07d8109aec4af7bdd09d84': 12,
 '0x7a6f424e3883f69688aa191a08e0f8b03a2da963afe6a080337ee476b9c3589e': 0,
 '0x05788e5a6fae013209599b7899cfc89f5559641feb121b7a3e6cc3be2daf4068': 2,
 '0x89bb1be1fb0ea747de9e02f21ed76c98a2c1a83775fc0fd9acc0d5e302d2b1ac': 18,
 '0xaf63a02c558a3f5b3dda2a040b0bdc78b8ab0e36b58eba524b3d29c04a34583b': 1,
 '0xd8b90b8c24e203c4c18e162b98d52ba01155af03e8d20a612e8aec24f14c4fc6': 1,
 '0x6a5ce53aa652306b139b77b3a0de38c6b1cd0e43383d6bdafc8f4a063527feb4': 3,
 '0xba563cc12e6ca57a4041b3959367cb0b09924cd19646031fe44678b675e769b0': 1,
 '0x7b08aa4b59c690330631aadebb59d83

In [11]:
ownership_tuples = []
for address, sig_to_count in cur_state.ownership.items():
    for sig, count in sig_to_count.items():
        if count > 0:
            ownership_tuples.append((sig, address, count))
ownership_tuples.sort()

indexed_tuples = []
index = 0
for (sig, address, count) in ownership_tuples:
    indexed_tuples.append((index, sig, address, count))
    index += 1
    
import csv
with open('data/ownership.csv', 'w') as f:
    writer = csv.writer(f)
    for t in indexed_tuples:
        writer.writerow(t)